In [138]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import v2 as trans2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import math

## Hyperparameters

In [139]:
epochs = 250
learning_rate = 0.0001
batch_size = 64


device = torch.device("mps")

## Load the Datasets and setup the Data Loaders (with data augmentation)

In [140]:
transform_normal = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor()
])

transform_train = transforms.Compose([
    transforms.Resize((250, 250)),
    # Randomly change the brightness of the image
    transforms.ColorJitter(brightness=0.5, contrast=0.2, saturation=0.2, hue=0.2),
    # Randomly flip the image horizontally
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically
    transforms.RandomRotation(55),  # Randomly rotate the image by 45 degrees
    transforms.ToTensor(),
    # transforms.Normalize(mean = mean, std = std) # Takes each value for the channel, subtracts the mean and divides by the standard deviation (value - mean) / std
])

# Define the transformations
transformations1 = transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((250, 250))])

# Load the dataset
training_dataset = torchvision.datasets.Flowers102(root='./data', split="train",
                                                   download=True, transform=transform_train)
testing_dataset = torchvision.datasets.Flowers102(root='./data', split="test",
                                                  download=True, transform=transformations1)
validation_dataset = torchvision.datasets.Flowers102(root='./data', split="val",
                                                     download=True, transform=transformations1)

# Create the dataloaders
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

## Create The Model

In [141]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.flatten = nn.Flatten()
    self.relu = nn.PReLU()
    
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(16),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(32),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(64),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(128),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer5 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer6 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    
    # self.fc1 = nn.Linear(256 * 3 * 3, 1024)
    self.fc1 = nn.Linear(256 * 3 * 3, 1024)
    self.drop = nn.Dropout(p=0.5)
    self.fc2 = nn.Linear(1024, 256)
    self.fc3 = nn.Linear(256, 102)  # Output layer for 102 classes

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.layer6(x)
    
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.drop(x)
    x = self.fc2(x)
    x = self.drop(x)
    x = self.fc3(x)
    # x = self.drop(x)
    # x = self.fc2(x)
    # x = self.relu(x)
    # x = self.fc3(x)
    return x


In [142]:
device = torch.device("mps")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=8)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

## Network Accuracy Tests (Validation and Testing)

In [143]:
def NetworkAccuracyOnValidation():
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        # num_class_correct = [0 for i in range(102)]
        # num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            # for i in range(len(labels)):
            #     label = labels[i]
            #     pred = predictions[i]
            #     if label == pred:
            #         num_class_correct[label] += 1
            #     num_class_samples[label] += 1

    val_epoch_loss = val_loss / val_total
    val_epoch_acc = 100. * val_correct / val_total
    
    return val_epoch_loss, val_epoch_acc

def NetworkAccuracyOnTesting():
    model.eval()
    total_correct = 0
    total_samples = 0
    num_class_correct = [0] * 102
    num_class_samples = [0] * 102
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predictions = outputs.max(1)
            total_samples += labels.size(0)
            total_correct += predictions.eq(labels).sum().item()
            
            c = (predictions == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                num_class_correct[label] += c[i].item()
                num_class_samples[label] += 1

            # for i in range(len(labels)):
            #     label = labels[i]
            #     pred = predictions[i]
            #     if (label == pred):
            #         num_class_correct[label] += 1
            #     num_class_samples[label] += 1

    acc = 100.0 * total_correct / total_samples
    print(f'Accuracy on testing set: {acc} %')

    for i in range(102):
        acc = 100.0 * num_class_correct[i] / num_class_samples[i]
        print(f'Accuracy of {i} : {acc} %')

## Training and Testing loop

In [144]:
best_accuracy_epoch = 0
best_accuracy = 0
early_stopping_patience = 15
no_improve_epochs = 0

for epoch in range(epochs):  # Loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader, 0):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        label_pred = model(images)
        loss = criterion(label_pred, labels)
        
        # Manually add L2 regularization
        l2_loss = 0
        for param in model.parameters():
            l2_loss += torch.sum(torch.pow(param, 2))
        loss += 0.01 * l2_loss  # L2 regularization term
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = label_pred.max(1)
        # predicted = torch.max(label_pred, 1)[1]
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        batch_corr = (predicted == labels).sum()
        batch_acc = batch_corr.item() / len(images)
        
        
        # print(f"Epoch Number {epoch}, Index = {i}/{len(train_loader)-1}, Loss = {loss.item()}")
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
    
    val_epoch_loss, val_epoch_acc = NetworkAccuracyOnValidation()
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')
    
    scheduler.step(val_epoch_loss)
    
    if (val_epoch_acc > best_accuracy):
        best_accuracy = val_epoch_acc
        best_accuracy_epoch = epoch
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch}")
            break
        
print(f"Best accuracy on validation split: {best_accuracy} at epoch {best_accuracy_epoch}")

model.load_state_dict(torch.load('best_model.pth'))
NetworkAccuracyOnTesting()


Epoch 1/250, Loss: 19.3096, Accuracy: 1.76%
Validation Loss: 4.6255, Validation Accuracy: 1.86%
Epoch 2/250, Loss: 19.0811, Accuracy: 2.06%
Validation Loss: 4.5399, Validation Accuracy: 2.06%
Epoch 3/250, Loss: 18.8083, Accuracy: 2.94%
Validation Loss: 4.3557, Validation Accuracy: 4.02%
Epoch 4/250, Loss: 18.5611, Accuracy: 3.33%
Validation Loss: 4.2014, Validation Accuracy: 5.39%
Epoch 5/250, Loss: 18.3123, Accuracy: 4.41%
Validation Loss: 4.0845, Validation Accuracy: 8.82%
Epoch 6/250, Loss: 18.1000, Accuracy: 5.98%
Validation Loss: 4.0449, Validation Accuracy: 8.63%
Epoch 7/250, Loss: 17.9143, Accuracy: 5.29%
Validation Loss: 3.9594, Validation Accuracy: 11.96%
Epoch 8/250, Loss: 17.7052, Accuracy: 6.08%
Validation Loss: 3.9017, Validation Accuracy: 9.51%
Epoch 9/250, Loss: 17.5219, Accuracy: 6.67%
Validation Loss: 3.8685, Validation Accuracy: 12.25%
Epoch 10/250, Loss: 17.3135, Accuracy: 6.47%
Validation Loss: 3.8237, Validation Accuracy: 13.53%
Epoch 11/250, Loss: 17.1250, Accurac

In [145]:
NetworkAccuracyOnTesting()

Accuracy on testing set: 52.34997560578956 %
Accuracy of 0 : 55.0 %
Accuracy of 1 : 65.0 %
Accuracy of 2 : 10.0 %
Accuracy of 3 : 27.77777777777778 %
Accuracy of 4 : 48.888888888888886 %
Accuracy of 5 : 32.0 %
Accuracy of 6 : 65.0 %
Accuracy of 7 : 78.46153846153847 %
Accuracy of 8 : 69.23076923076923 %
Accuracy of 9 : 80.0 %
Accuracy of 10 : 14.925373134328359 %
Accuracy of 11 : 56.71641791044776 %
Accuracy of 12 : 72.41379310344827 %
Accuracy of 13 : 78.57142857142857 %
Accuracy of 14 : 44.827586206896555 %
Accuracy of 15 : 33.333333333333336 %
Accuracy of 16 : 67.6923076923077 %
Accuracy of 17 : 20.967741935483872 %
Accuracy of 18 : 27.586206896551722 %
Accuracy of 19 : 55.55555555555556 %
Accuracy of 20 : 60.0 %
Accuracy of 21 : 76.92307692307692 %
Accuracy of 22 : 40.84507042253521 %
Accuracy of 23 : 45.45454545454545 %
Accuracy of 24 : 80.95238095238095 %
Accuracy of 25 : 47.61904761904762 %
Accuracy of 26 : 75.0 %
Accuracy of 27 : 58.69565217391305 %
Accuracy of 28 : 67.24137931